In [8]:
%load_ext autoreload
%autoreload 2

import json

from google import genai
import opencc

from ai_storyteller.music_generation.diffrhythm import DiffRhythm
from ai_storyteller.utils.get_env_var import get_env_var

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
api_key = get_env_var("GEMINI_API_KEY")
client = genai.Client(api_key=api_key)
model = "gemini-2.0-flash"

t2s = opencc.OpenCC("t2s.json")

Found GEMINI_API_KEY using strategy: _get_from_env


In [10]:
dr = DiffRhythm()

DiffRhythm package path set to: vendor/DiffRhythm
Using DiffRhythm shell script: vendor/DiffRhythm/scripts/run_diffrhythm.sh


https://discoverpods.com/overused-royalty-free-songs/

In [11]:
dr.generate_music(
    ref_audio_path="../../data/music_generation/music/snoozy beats - Feel the Glow.mp3",
    output_dir=".",
    output_file_name="children_music.wav"
)

LRC path not provided, using script default: data/music_generation/lrc/eg_en_full.lrc
Executing shell script: /bin/bash /home/richard/github/ai-storyteller/vendor/DiffRhythm/scripts/run_diffrhythm.sh --lrc-path /home/richard/github/ai-storyteller/data/music_generation/lrc/eg_en_full.lrc --ref-audio-path /home/richard/github/ai-storyteller/data/music_generation/music/snoozy beats - Feel the Glow.mp3 --chunked --audio-length 95 --repo-id ASLP-lab/DiffRhythm-full --output-dir /home/richard/github/ai-storyteller/notebooks/music_generation --output-file-name children_music.wav

--- Shell Script Execution Successful ---
stdout:
 Changed working directory to: /home/richard/github/ai-storyteller/vendor/DiffRhythm
Using provided LRC path: /home/richard/github/ai-storyteller/data/music_generation/lrc/eg_en_full.lrc
Using provided reference audio path: /home/richard/github/ai-storyteller/data/music_generation/music/snoozy beats - Feel the Glow.mp3
Final Configuration:
  LRC Path:       /home/rich

PosixPath('/home/richard/github/ai-storyteller/notebooks/music_generation/children_music.wav')

# Generate lyrics from a children's story

In [4]:
with open("../../data/stories/the_wolf_and_the_seven_kids/story.json") as f:
    data = json.load(f)
data

{'title': '大野狼與七隻小羊',
 'style_guide': {'overall_style': 'Warm, friendly, cartoon style with soft edges and bright, cheerful colors. Resembles classic European fairy tale illustrations but slightly more modern and simplified. Focus on expressive characters. Use a watercolor texture effect.',
  'character_design': {'mother_goat': 'Kind face, wearing an apron.',
   'little_kids': 'Cute, fluffy, varied slightly in appearance (maybe one has spots, one has floppy ears).',
   'wolf': 'Cunning but slightly bumbling appearance, dark fur, sharp teeth (but not overly terrifying for children).',
   'baker_miller': 'Traditional European peasant attire.'},
  'backgrounds': 'Softly detailed, slightly whimsical settings (cozy cottage, lush green forest edge, rustic village shops, sunny meadow).'},
 'pages': {'cover': {'image_prompt': "Generate a children's book cover image in a warm, friendly, cartoon style with soft edges, bright cheerful colors, and a watercolor texture effect, using the defined cha

In [5]:
pages = data["pages"]
text = []
for page in pages:
    text.append(pages[page]["text"].strip())
text = "\n".join(text).strip()
print(text)

從前從前，有羊媽媽和七隻小羊。
媽媽要去森林，叮嚀孩子們：「要小心大野狼！牠聲音粗啞，腳是黑色的！」

沒多久，叩叩叩！
一個粗啞的聲音說：「開門啊，媽媽回來了！」
小羊們大叫：「不開！你是大野狼！」
大野狼吃了粉筆，又來敲門，聲音變細了。
但牠把黑爪子放在窗邊。

小羊們又叫：「不開！媽媽的腳不是黑色的！」

於是大野狼去找麵包師傅和磨坊主人，把腳抹上麵糰和麵粉弄白。
壞蛋第三次來敲門，伸進白色的腳。
小羊們相信了，把門打開。
結果衝進來的是大野狼！
小羊們嚇壞了，趕快躲到桌下、床上、爐子裡、櫥櫃裡、洗臉盆下和鐘殼裡！
大野狼把躲起來的小羊一個個找出來，吞進肚子裡。
只有躲在鐘殼裡最小的沒被找到。

吃飽後，牠走到外面草地上，躺在樹下睡著了。
羊媽媽回來，看到家裡亂七八糟，孩子都不見了，傷心地哭了起來。

這時，最小的羊從鐘殼裡叫著：「媽媽，我在這裡！大野狼把哥哥姐姐們都吃掉了！」
媽媽帶著小羊找到正在打呼的大野狼，看到牠肚子裡有東西在動！

媽媽趕快拿來剪刀，小心地剪開狼肚子，六隻小羊一個接一個跳了出來，全都平安無事！
牠們高興地抱著媽媽。
媽媽說：「快去找些大石頭來！」
小羊們把石頭塞滿大野狼的肚子，媽媽再趕快把牠的肚子縫起來。

大野狼醒來覺得好渴，走到井邊彎腰喝水時，重重的石頭讓牠一頭栽進井裡淹死了！

七隻小羊看到了，高興地跑來大叫：「大野狼死了！」
牠們和媽媽一起圍著水井，開心地跳起舞來。


## Prepare prompt for lyric generation

In [ ]:
gen_lyrics_from_story_prompt = """\
根據以下故事生成歌詞，這首歌長度為{seconds}秒
格式應該是 .lrc，例如：

```
[00:00.00]歌詞內容
[00:01.00]歌詞內容
...
```
除了歌詞以外，其他的內容都不需要，中間不能有空行，如果故事的內容是中文

故事：
```
{story}
```
"""

filled_gen_lyrics_from_story_prompt = gen_lyrics_from_story_prompt.format(story=text, seconds=95)
print(filled_gen_lyrics_from_story_prompt)

根據以下故事生成歌詞，這首歌長度為95秒
格式應該是 .lrc，例如：

```
[00:00.00]歌詞內容
[00:01.00]歌詞內容
...
```
除了歌詞以外，其他的內容都不需要，中間不能有空行，如果故事的內容是中文，請輸出簡體字的歌詞

故事：
```
從前從前，有羊媽媽和七隻小羊。
媽媽要去森林，叮嚀孩子們：「要小心大野狼！牠聲音粗啞，腳是黑色的！」

沒多久，叩叩叩！
一個粗啞的聲音說：「開門啊，媽媽回來了！」
小羊們大叫：「不開！你是大野狼！」
大野狼吃了粉筆，又來敲門，聲音變細了。
但牠把黑爪子放在窗邊。

小羊們又叫：「不開！媽媽的腳不是黑色的！」

於是大野狼去找麵包師傅和磨坊主人，把腳抹上麵糰和麵粉弄白。
壞蛋第三次來敲門，伸進白色的腳。
小羊們相信了，把門打開。
結果衝進來的是大野狼！
小羊們嚇壞了，趕快躲到桌下、床上、爐子裡、櫥櫃裡、洗臉盆下和鐘殼裡！
大野狼把躲起來的小羊一個個找出來，吞進肚子裡。
只有躲在鐘殼裡最小的沒被找到。

吃飽後，牠走到外面草地上，躺在樹下睡著了。
羊媽媽回來，看到家裡亂七八糟，孩子都不見了，傷心地哭了起來。

這時，最小的羊從鐘殼裡叫著：「媽媽，我在這裡！大野狼把哥哥姐姐們都吃掉了！」
媽媽帶著小羊找到正在打呼的大野狼，看到牠肚子裡有東西在動！

媽媽趕快拿來剪刀，小心地剪開狼肚子，六隻小羊一個接一個跳了出來，全都平安無事！
牠們高興地抱著媽媽。
媽媽說：「快去找些大石頭來！」
小羊們把石頭塞滿大野狼的肚子，媽媽再趕快把牠的肚子縫起來。

大野狼醒來覺得好渴，走到井邊彎腰喝水時，重重的石頭讓牠一頭栽進井裡淹死了！

七隻小羊看到了，高興地跑來大叫：「大野狼死了！」
牠們和媽媽一起圍著水井，開心地跳起舞來。
```



## Generate

In [ ]:
res = client.models.generate_content(
    model=model,
    contents=filled_gen_lyrics_from_story_prompt,
)
lyrics =res.text
print(lyrics)

[00:01.02]從前從前 羊媽媽
[00:04.28]七隻小羊 在家
[00:07.41]森林很遠 要小心呀
[00:10.49]大野狼 壞傢伙

[00:13.92]粗啞聲音 黑黑腳丫
[00:17.07]別開門呀 別相信它
[00:20.27]叩叩叩 誰在敲打
[00:23.39]騙人的吧 不聽不聽

[00:26.90]粉筆麵糰 白腳丫呀
[00:29.95]小羊開門 大狼來啦
[00:33.17]躲起來呀 快躲好呀
[00:36.28]桌下床上 爐子裡呀

[00:39.83]一個一個 被它吃啦
[00:42.98]鐘殼最小 躲過一劫
[00:46.10]媽媽回來 家亂糟糟
[00:49.24]孩子不見 傷心哭嚎

[00:52.73]鐘殼裡面 最小的叫
[00:55.85]大野狼啊 吃掉哥哥
[00:59.01]狼在睡覺 肚子鼓鼓
[01:02.16]剪刀石頭 媽媽來救

[01:05.71]剪開肚子 羊羊跳出
[01:08.86]全都平安 真好真好
[01:12.05]石頭塞滿 狼的肚子
[01:15.15]媽媽縫好 肚子鼓鼓

[01:18.66]口渴醒來 井邊喝水
[01:21.81]頭重腳輕 掉進井裡
[01:24.97]大野狼啊 淹死啦
[01:28.11]羊羊歡呼 快樂歌唱

[01:31.61]大野狼死啦 死啦死啦
[01:34.77]媽媽羊羊 跳舞歌唱
[01:37.93]大野狼死啦 死啦死啦
[01:41.07]媽媽羊羊 永遠幸福
[01:44.07]



## Simplified Chinese seems to work better, so we'll convert

In [ ]:
t2s.convert(lyrics)

NameError: name 'res' is not defined

In [15]:
with open("/tmp/lyrics.lrc", "w") as f:
    f.write(res.text)

In [14]:
dr.generate_music(
    # lrc_path="../../data/music_generation/lrc/test.lrc",
    lrc_path="/tmp/lyrics.lrc",
    prompt="silent movie",
    # ref_audio_path="../../data/music_generation/music/Amazing Plan - Distressed.mp3",
    output_dir=".",
    audio_length=95
)

Executing shell script: /bin/bash /home/richard/github/ai-storyteller/vendor/DiffRhythm/scripts/run_diffrhythm.sh --lrc-path /tmp/lyrics.lrc --ref-prompt silent movie --chunked --audio-length 95 --repo-id ASLP-lab/DiffRhythm-full --output-dir /home/richard/github/ai-storyteller/notebooks/music_generation --output-file-name output.wav

--- Shell Script Execution Successful ---
stdout:
 Changed working directory to: /home/richard/github/ai-storyteller/vendor/DiffRhythm
Using provided LRC path: /tmp/lyrics.lrc
Using provided reference prompt: silent movie
Final Configuration:
  LRC Path:       /tmp/lyrics.lrc
  Ref Prompt:     silent movie
  Ref Audio Path: 
  Audio Length:   95
  Repo ID:        ASLP-lab/DiffRhythm-full
  Output Dir:     /home/richard/github/ai-storyteller/notebooks/music_generation
  Output File:    output.wav
  Chunked:        true

Executing Python script...
Current working directory: /home/richard/github/ai-storyteller/vendor/DiffRhythm
./g2p/sources/g2p_chinese_mode

PosixPath('/home/richard/github/ai-storyteller/notebooks/music_generation/output.wav')

# Using prompt seems to work better than using reference song

# Make lyrics fit the timings using example `.lrc`

# Generate instruments only 

# Identify instruments

# Figure out the prompt used to style the song